<a href="https://colab.research.google.com/github/Lisa-Ho/100DaysOfDataviz/blob/main/Test_Countline_Speeds_API_ATC_Report_Generator_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Countline Speeds API ATC Report Generator


## Getting Started
Let's begin by importing the packages we'll need!

In [ ]:
import requests
import getpass
import json
from datetime import date, datetime, timedelta
import csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Import
At the end of this process we will have a `dict` called `data` with the response from out Countline Speed API.
### Countline Details
Choose one or more coutlines, the classes you want data for and the date. Other parameters are set automatically.

In [ ]:
#@title Countline Details { run: "auto", vertical-output: true }
countline_ids = "40284,40285,40286" #@param {type:"string"}
classes = "car,taxi,van,bus,rigid,motorbike,cyclist,pedestrian" #@param {type:"string"}
date = "2022-02-01" #@param {type:"date"}

params = {}
params['countline_ids'] = countline_ids
params['classes'] = classes
params['from'] = datetime.fromisoformat(date).strftime('%Y-%m-%dT%H:%M:%S.000Z')
params['to'] = (datetime.fromisoformat(date)+timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S.000Z')
params['time_bucket'] = "15m"
params['speed_bucket_number'] = "66"
params['max_speed'] = "44"
params['min_speed'] = "0"
params['fill_zeros'] = "true"

### Authentication
Enter your username and password:

In [ ]:
auth_body = {}
auth_body['username'] = "countline-telemetry-testing-api-user"
auth_body['password'] = getpass.getpass()

··········


A token needs to be generated every 5 mins.

In [ ]:
auth_response = requests.post("https://api.vivacitylabs.com/get-token", data=auth_body, headers={'Content-Type':'application/x-www-form-urlencoded'})
headers = {}
headers['Authorization'] = "Bearer " + auth_response.json()['access_token']

### Getting the data

Here's the API call to get the countline speed data.

In [ ]:
response = requests.get("https://beta.api.vivacitylabs.com/countline/speed", params=params, headers=headers)
print(response.url)
print(str(response.status_code) + ": " + response.text)
data = response.json()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Data Processing
And now we process that to output a `dict` keyed off countlineID. The associated value is an array of arrays that can be written to a csv. Currently export outputs to a specific set of classes in a specific order, but the `row` variable can be adjusted to output different classes if required. Note the factor `2.236936` used to covert from m/s to mph.

In [ ]:
export = []
for countline, countlineData in data.items():
  directionData = {}
  for timeData in countlineData:
    startTime = datetime.strptime(timeData['from'],'%Y-%m-%dT%H:%M:%S.000Z')
    endTime = datetime.strptime(timeData['to'],'%Y-%m-%dT%H:%M:%S.000Z')
    directionData['out'] = timeData['anti_clockwise']
    directionData['in'] = timeData['clockwise']
    counts = {}
    averages = {}
    for directionLabel, classData in directionData.items():
      counts[directionLabel] = {}
      averages[directionLabel] = {}
      for classLabel, speedData in directionData[directionLabel].items():
        countTotal = 0
        speedTotal = 0
        for speed, data in speedData.items():
          countTotal += data
          speedTotal += data*float(speed)
          if countTotal != 0:
            counts[directionLabel][classLabel] = str(countTotal)
            averages[directionLabel][classLabel] = speedTotal*2.236936/countTotal
          else:
            counts[directionLabel][classLabel] = '0'
            averages[directionLabel][classLabel] = 'n/a'
      row = [str(startTime.date()),
             str(startTime.time()),
             str(endTime.time()),
             countline,
             directionLabel,
             averages[directionLabel]['car'],
             averages[directionLabel]['taxi'],
             averages[directionLabel]['van'],
             averages[directionLabel]['bus'],
             averages[directionLabel]['rigid'],
             averages[directionLabel]['motorbike'],
             averages[directionLabel]['cyclist'],
             averages[directionLabel]['pedestrian'],
             counts[directionLabel]['car'],
             counts[directionLabel]['taxi'],
             counts[directionLabel]['van'],
             counts[directionLabel]['bus'],
             counts[directionLabel]['rigid'],
             counts[directionLabel]['motorbike'],
             counts[directionLabel]['cyclist'],
             counts[directionLabel]['pedestrian']]
      export.append(row)

In [ ]:
print(export)

[['2022-02-01', '00:00:00', '00:15:00', '40284', 'anti_clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:00:00', '00:15:00', '40284', 'clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:15:00', '00:30:00', '40284', 'anti_clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:15:00', '00:30:00', '40284', 'clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:30:00', '00:45:00', '40284', 'anti_clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:30:00', '00:45:00', '40284', 'clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:45:00', '01:00:00', '40284', 'anti_clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '00:45:00', '01:00:00', '40284', 'clockwise', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a'], ['2022-02-01', '01:00:00', '01:15:0

## Data Export
Now let's write this to a .csv.

In [ ]:
header = ['date',	'start_time',	'end_time', 'countline_id', 'direction',	'car (mph)',	'taxi (mph)',	'LGV (mph)',	'bus (mph)',	'OGV (mph)',	'motorbike (mph)',	'cyclist (mph)',	'pedestrian (mph)', 'car count', 'taxi count', 'LGV count', 'bus count', 'OGV count', 'motorbike count', 'cyclist count', 'pedestrian count']
with open('/content/drive/My Drive/Downloads/' + 'test' + '.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  writer.writerows(export)